In [1]:
%matplotlib widget
from sklearn import datasets, manifold
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy import stats

In [17]:
n_samples = 500
x, y = datasets.make_s_curve( n_samples=n_samples )

idx = y.argsort()
y.sort()
x = x[idx]

# for i in range( n_samples ):
#     if i < n_samples / 2:
#         y[i] = -2.0
#     else:
#         y[i] = 2.0

In [18]:
x = np.concatenate( (x[:300], x[300:]) )
y = np.concatenate( (y[:300], y[300:]) )

In [19]:
fig = plt.figure()
ax = fig.add_subplot( 111, projection='3d' )

x_vals = x[:, 0]
z_vals = x[:, 1]
y_vals = x[:, 2]

ax.scatter( x_vals, y_vals, z_vals, c=y )
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
def fit_plot( x, y, n_neighbors, n_components ):
    isomap = manifold.Isomap( n_neighbors=n_neighbors, n_components=n_components )
    y_isomap = isomap.fit_transform( x )
    
    lle = manifold.LocallyLinearEmbedding( n_neighbors=n_neighbors, n_components=n_components )
    y_lle = lle.fit_transform( x )
    
    laplace = manifold.SpectralEmbedding( n_neighbors=n_neighbors, n_components=n_components )
    y_laplace = laplace.fit_transform( x )
    
    ltsa = manifold.LocallyLinearEmbedding( n_neighbors=n_neighbors, n_components=n_components, method='ltsa' )
    y_ltsa = ltsa.fit_transform( x )
    
    fig, (ax1, ax2, ax3, ax4 ) = plt.subplots( 1, 4 )
    
    x1_vals = y_isomap[:, 0]
    y1_vals = y_isomap[:, 1]
    ax1.scatter( x1_vals, y1_vals, c=y )
    
    x2_vals = y_lle[:, 0]
    y2_vals = y_lle[:, 1] 
    ax2.scatter( x2_vals, y2_vals, c=y )
    
    x3_vals = y_laplace[:, 0]
    y3_vals = y_laplace[:, 1]
    ax3.scatter( x3_vals, y3_vals, c=y )
    
    x4_vals = y_ltsa[:, 0]
    y4_vals = y_ltsa[:, 1]
    ax4.scatter( x4_vals, y4_vals, c=y )
    
    plt.plot()
    return (y_isomap, y_lle, y_laplace, y_ltsa)
    
def residual_variance( dx, dy ):
    corr = stats.spearmanr( dx, dy, axis=1 ).correlation
    return corr

In [16]:
y_isomap, y_lle, y_laplace, y_ltsa = fit_plot( x, y, n_neighbors=10, n_components=2 )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …